In [1]:
import random
import statistics
import numpy as np

def generate_turtle_reserves(num_turtles=1000, low=50, high=350):
    """
    Generate random reserve prices for the Turtles.
    For simplicity, use uniform distribution here; adjust as desired.
    """
    return [random.randint(low, high) for _ in range(num_turtles)]

def generate_competitors(num_competitors=1000):
    """
    Generate a list of (first_bid, second_bid) for competitor traders.
    You can customize to produce 'clusters' of strategies or random.
    Below: a simple example that randomizes each competitor's bids
    within a plausible range.
    """
    competitors = []
    for _ in range(num_competitors):
        b1 = random.randint(200, 320)    # random first bid
        b2 = random.randint(b1, 325)     # second bid >= first, possibly up to 325
        competitors.append((b1, b2))
    return competitors

def simulate_round(
    turtle_reserves,
    competitors,
    your_first_bid,
    your_second_bid,
    resale_price=320
):
    """
    Run the two-round simulation and return the total profit you make.
    
    Parameters:
    -----------
    turtle_reserves : list of ints
        The reserve price for each turtle.
    competitors : list of (first_bid, second_bid)
        The list of competitor strategies.
    your_first_bid, your_second_bid : int
        Your strategy bids.
    resale_price : int
        The price you can sell Flippers for after purchase.
    
    Returns:
    --------
    profit : float
        Your total profit from all Turtles you manage to buy.
    bought_count : int
        Number of Turtles you bought.
    """
    n_comp = len(competitors)
    n_turtles = len(turtle_reserves)
    
    # Combine your strategy into the list so we can treat "all traders" uniformly
    # We'll treat you as trader index 0, then the competitors as 1..n_comp
    # For easy reference, we keep them in a single list. 
    # Format: all_traders[i] = (first_bid, second_bid)
    all_traders = [(your_first_bid, your_second_bid)] + competitors
    
    # ROUND 1: 
    # Each turtle picks the lowest first-bid >= reserve (if any).
    
    # We'll store which trader (if any) the Turtle sold to in round 1,
    # and at what price. If not sold, store None.
    round1_buyer = [None]*n_turtles
    round1_price = [None]*n_turtles
    
    # We iterate over each Turtle, check all traders' first bids
    # to find the minimal bid >= turtle_reserve.
    for t_idx in range(n_turtles):
        reserve = turtle_reserves[t_idx]
        # find all traders whose first_bid >= reserve
        # pick the one with the smallest such first_bid
        possible_buyers = []
        for trader_idx, (fbid, _) in enumerate(all_traders):
            if fbid >= reserve:
                possible_buyers.append((fbid, trader_idx))
        
        if len(possible_buyers) > 0:
            # sort by first-bid ascending
            possible_buyers.sort(key=lambda x: x[0])
            chosen_price, chosen_trader = possible_buyers[0]
            round1_buyer[t_idx] = chosen_trader
            round1_price[t_idx] = chosen_price
    
    # Turtles that remain unsold after Round 1 (buyer=None) proceed to Round 2
    # Round 2 condition: second_bid must be > average( second_bids ) of *all* traders
    # and >= reserve. Turtle picks the *lowest second_bid* among those that qualify.
    
    # 1) Compute the archipelago-wide average second bid
    second_bids_all = [t[1] for t in all_traders]  # (your_second_bid + all competitors)
    avg_second_bid = statistics.mean(second_bids_all)
    
    # We'll store final buyer (if any) and price for each Turtle after Round 2
    final_buyer = [round1_buyer[t] for t in range(n_turtles)]
    final_price = [round1_price[t] for t in range(n_turtles)]
    
    for t_idx in range(n_turtles):
        if round1_buyer[t_idx] is None:  # not sold in Round 1
            reserve = turtle_reserves[t_idx]
            
            # Which traders have second_bid >= reserve *and*
            # second_bid > avg_second_bid?
            # Among those, pick the lowest second_bid
            possible_buyers_round2 = []
            for trader_idx, (_, sbid) in enumerate(all_traders):
                if sbid >= reserve and sbid > avg_second_bid:
                    possible_buyers_round2.append((sbid, trader_idx))
            
            if len(possible_buyers_round2) > 0:
                possible_buyers_round2.sort(key=lambda x: x[0])  # sort by second_bid
                chosen_price, chosen_trader = possible_buyers_round2[0]
                final_buyer[t_idx] = chosen_trader
                final_price[t_idx] = chosen_price
    
    # Now compute how many Turtles YOU got and total profit
    your_profit = 0
    your_count = 0
    
    for t_idx in range(n_turtles):
        if final_buyer[t_idx] == 0:  # means you are the buyer
            buy_price = final_price[t_idx]
            # Profit is resale - buy_price
            your_profit += (resale_price - buy_price)
            your_count += 1
    
    return your_profit, your_count

def run_simulation_example(
    num_turtles=1000,
    num_competitors=2000,
    your_first_bid=300,
    your_second_bid=315,
    trials=100
):
    """
    Run multiple simulation trials to estimate average profit 
    for a given strategy vs. random competitors.
    """
    total_profit = 0
    total_count = 0
    for _ in range(trials):
        # Generate random turtle reserves
        turtle_reserves = generate_turtle_reserves(num_turtles, 50, 350)
        # Generate random competitor strategies
        comp_strategies = generate_competitors(num_competitors)
        # Simulate
        profit, count = simulate_round(
            turtle_reserves,
            comp_strategies,
            your_first_bid,
            your_second_bid
        )
        total_profit += profit
        total_count += count
    
    avg_profit = total_profit / trials
    avg_count = total_count / trials
    return avg_profit, avg_count



if __name__ == "__main__":
    # Example usage:
    # Let's see how your strategy (300, 315) performs in 100 trials
    avg_profit, avg_count = run_simulation_example(
        num_turtles=1000,
        num_competitors=5000,
        your_first_bid=300,
        your_second_bid=315,
        trials=100
    )
    print(f"Average profit over 100 trials: {avg_profit:.2f}")
    print(f"Average flippers bought over 100 trials: {avg_count:.1f}")


Average profit over 100 trials: 65.60
Average flippers bought over 100 trials: 3.3


In [2]:
import random
import statistics
import numpy as np

def generate_turtle_reserves(num_turtles=1000, low=50, high=350):
    """Same as before. Or customize distribution."""
    return [random.randint(low, high) for _ in range(num_turtles)]

def simulate_round_extended(
    turtle_reserves,
    # Now we pass a *list* of all strategies: competitor_strategies + your_strategy
    all_strategies,  # list of (first_bid, second_bid)
    resale_price=320
):
    """
    Extended version of the 2-round simulation.
    Returns: 
      - profits array, where profits[i] is the total profit of trader i
      - number of Turtles each trader bought
      - the final average second-bid
    """
    n_traders = len(all_strategies)
    n_turtles = len(turtle_reserves)
    
    # Round 1
    round1_buyer = [None]*n_turtles
    round1_price = [None]*n_turtles
    
    for t_idx in range(n_turtles):
        reserve = turtle_reserves[t_idx]
        possible_buyers = []
        for trader_idx, (fbid, _) in enumerate(all_strategies):
            if fbid >= reserve:
                possible_buyers.append((fbid, trader_idx))
        
        if possible_buyers:
            possible_buyers.sort(key=lambda x: x[0])  # pick smallest feasible bid
            chosen_price, chosen_trader = possible_buyers[0]
            round1_buyer[t_idx] = chosen_trader
            round1_price[t_idx] = chosen_price
    
    # Round 2
    second_bids = [sbid for (_, sbid) in all_strategies]
    avg_second_bid = statistics.mean(second_bids)
    
    final_buyer = [round1_buyer[t] for t in range(n_turtles)]
    final_price = [round1_price[t] for t in range(n_turtles)]
    
    for t_idx in range(n_turtles):
        if round1_buyer[t_idx] is None:  # not sold in Round 1
            reserve = turtle_reserves[t_idx]
            possible_buyers_2 = []
            for trader_idx, (_, sbid) in enumerate(all_strategies):
                if sbid >= reserve and sbid > avg_second_bid:
                    possible_buyers_2.append((sbid, trader_idx))
            if possible_buyers_2:
                possible_buyers_2.sort(key=lambda x: x[0])
                chosen_price, chosen_trader = possible_buyers_2[0]
                final_buyer[t_idx] = chosen_trader
                final_price[t_idx] = chosen_price
    
    # Compute profits
    profits = [0]*n_traders
    bought_counts = [0]*n_traders
    
    for t_idx in range(n_turtles):
        buyer = final_buyer[t_idx]
        if buyer is not None:
            buy_price = final_price[t_idx]
            profits[buyer] += (resale_price - buy_price)
            bought_counts[buyer] += 1
    
    return profits, bought_counts, avg_second_bid


# Genetic algorithm parameters
POP_SIZE = 200   # number of competitor strategies in each generation
MUTATION_RATE = 0.1
NUM_GENERATIONS = 50
NUM_TURTLES = 1000

def random_strategy():
    """Generate a random (first_bid, second_bid) within a range."""
    fbid = random.randint(200, 320)
    sbid = random.randint(fbid, 325)
    return (fbid, sbid)

def mutate_strategy(strategy):
    """Mutate a strategy with some small random changes."""
    (fbid, sbid) = strategy
    if random.random() < 0.5:
        # mutate first_bid
        fbid += random.randint(-5,5)
    else:
        # mutate second_bid
        sbid += random.randint(-5,5)
    # clamp
    fbid = max(1, min(325, fbid))
    sbid = max(fbid, min(330, sbid))  # ensure second bid >= first bid
    return (fbid, sbid)

def crossover(strat1, strat2):
    """
    Example "crossover" that picks the first_bid from one 
    and second_bid from the other. 
    This is very simplistic; you can design more nuanced crossovers.
    """
    fbid = random.choice([strat1[0], strat2[0]])
    sbid = random.choice([strat1[1], strat2[1]])
    if sbid < fbid:
        sbid = fbid
    return (fbid, sbid)

# Initialize population
population = [random_strategy() for _ in range(POP_SIZE)]

your_strategy = (300, 315)  # You can fix or let it also evolve

for gen in range(NUM_GENERATIONS):
    # For each generation, we'll do:
    # 1) Evaluate each strategy in the population
    # 2) Select top performers
    # 3) Reproduce/mutate

    # We'll run the simulation with all these strategies + your strategy
    # But here, let's assume *everyone* in the population is a competitor,
    # so the set of all_traders is the population + your strategy (or not, depending on your design).
    
    turtle_reserves = generate_turtle_reserves(NUM_TURTLES)
    
    # Evaluate each competitor individually:
    # That means we run a separate simulation for each competitor strategy
    # with the rest of the population? 
    # Or do we do "all compete simultaneously"? Typically, in a GA, all 200 strategies 
    # are "in the game" simultaneously, each representing 1 agent. 
    # So let's do "all compete simultaneously."

    # We'll treat the entire population as the set of strategies in the game
    all_strategies = population[:]  # shallow copy
    # Optionally, you can also add your_strategy, so we have +1 trader
    # but let's assume it's just the population for an equilibrium scenario
    profits, bought_counts, avg_second = simulate_round_extended(
        turtle_reserves, all_strategies
    )
    
    # Now we have profits[i] for strategy i
    # Sort strategies by profit
    strat_with_fitness = list(zip(population, profits))
    strat_with_fitness.sort(key=lambda x: x[1], reverse=True)  # sort descending profit
    
    # Keep top fraction
    top_fraction = 0.3
    cutoff = int(top_fraction * POP_SIZE)
    top_strats = strat_with_fitness[:cutoff]
    
    # Prepare next generation
    new_population = []
    # Always keep the top few
    for (s, _) in top_strats:
        new_population.append(s)
    
    # Fill the rest by crossover + mutation among top strats
    while len(new_population) < POP_SIZE:
        parent1, _ = random.choice(top_strats)
        parent2, _ = random.choice(top_strats)
        child = crossover(parent1, parent2)
        # mutation
        if random.random() < MUTATION_RATE:
            child = mutate_strategy(child)
        new_population.append(child)
    
    population = new_population
    
    if (gen+1) % 5 == 0:  # Print some debug info every 5 generations
        avg_pop_profit = np.mean([p for (_, p) in strat_with_fitness])
        best_strat, best_profit = strat_with_fitness[0]
        print(f"Generation {gen+1}: best={best_strat}, best_profit={best_profit:.2f}, avg_profit={avg_pop_profit:.2f}, avg_second={avg_second:.2f}")

# After final generation
# "population" hopefully is near some stable set of bids.
# We can examine them:
print("\nFinal population sample:")
for i in range(10):
    print(population[i])


Generation 5: best=(199, 257), best_profit=58927.00, avg_profit=410.75, avg_second=283.75
Generation 10: best=(199, 257), best_profit=57596.00, avg_profit=413.32, avg_second=282.64
Generation 15: best=(197, 303), best_profit=61568.00, avg_profit=422.81, avg_second=283.23
Generation 20: best=(197, 303), best_profit=57823.00, avg_profit=418.37, avg_second=284.75
Generation 25: best=(197, 303), best_profit=58705.00, avg_profit=421.37, avg_second=287.56
Generation 30: best=(196, 257), best_profit=59768.00, avg_profit=417.10, avg_second=284.71
Generation 35: best=(195, 306), best_profit=65626.00, avg_profit=450.74, avg_second=286.25
Generation 40: best=(195, 306), best_profit=63806.00, avg_profit=446.31, avg_second=292.51
Generation 45: best=(195, 306), best_profit=59681.00, avg_profit=432.72, avg_second=288.65
Generation 50: best=(192, 306), best_profit=60046.00, avg_profit=443.73, avg_second=290.00

Final population sample:
(192, 306)
(214, 228)
(224, 290)
(210, 293)
(207, 264)
(199, 257)